In [1]:
import re
import string
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression as lr
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import random as rnd
from sklearn.model_selection import train_test_split 
import string
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs


df=pd.read_json("D:\B.E\sem7\Project work stage1\cyberbullying\Dataset for Detection of Cyber-Trolls.json",lines=True)

In [3]:
df["label"] = df.annotation.apply(lambda x: x.get('label'))
df["label"] = df.label.apply(lambda x: x[0])
df['label']=df['label'].astype(int)
df.drop(['extras'],inplace=True,axis=1)
df.head()

,content,annotation,label
0,Get fucking real dude.,"{'notes': '', 'label': ['1']}",1
1,She is as dirty as they come and that crook ...,"{'notes': '', 'label': ['1']}",1
2,why did you fuck it up. I could do it all day...,"{'notes': '', 'label': ['1']}",1
3,Dude they dont finish enclosing the fucking s...,"{'notes': '', 'label': ['1']}",1
4,WTF are you talking about Men? No men thats n...,"{'notes': '', 'label': ['1']}",1


In [4]:
x=df["content"]
y=df["label"]


In [ ]:
Python
# Open the file for writing
F = open('list.txt', 'w')

# The reason why we used a list comprehension 
# is that we wanted to convert the numbers to 
# strings so that they can be written to disk
F.writelines([str(x) + "\n" for x in x])

# Close the file
F.close()

In [66]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42,shuffle=True)
print("X_train:",len(x_train))
print("X_test:",len(x_test))
print("y_train",len(y_train))
print("y_test",len(y_test))


X_train: 13400
X_test: 6600
y_train 13400
y_test 6600


In [12]:
freqs = build_freqs(x_train,y_train)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 14318


In [16]:
print('This is an example of a positive tweet: \n', x_train[7070])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(x_train[7070]))

This is an example of a positive tweet: 
 HAHAHHAHAAHHAHAHA LULZ   @Mod_Alex  your STILL a cunt!!!!! hahahahahahahah lulz lulz lulz lulz lulz

This is an example of the processed version of the tweet: 
 ['hahahhahaahhahaha', 'lulz', 'still', 'cunt', 'hahahahahahahah', 'lulz', 'lulz', 'lulz', 'lulz', 'lulz']


In [27]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
    assert(x.shape == (1, 3))
    return x

In [35]:
extract_features(x_train[17244],freqs)

array([[1.000e+00, 1.043e+03, 1.854e+03]])

In [68]:
X = np.zeros((len(x_train), 3))
z=0
for i in x_train:
    X[z, :]= extract_features(i, freqs)
    z+=1

# training labels corresponding to X
Y = y_train
print(X[0][1])

309.0


In [75]:
print(Y)

7070     1
13010    0
6688     1
9976     0
5175     1
        ..
11284    0
11964    0
5390     1
860      1
15795    0
Name: label, Length: 13400, dtype: int32


In [103]:
log_clf = LogisticRegression(C=89,random_state=0)
rnd_clf = RandomForestClassifier(n_estimators=800, random_state=0,max_features=6)
svm_clf = SVC(gamma="auto", random_state=0,)
knn_clf=KNeighborsClassifier(metric="euclidean",n_neighbors=7)
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('knn', knn_clf),('svc',svm_clf)],
    voting='hard')


In [104]:
voting_clf.fit(X,Y)

VotingClassifier(estimators=[('lr', LogisticRegression(C=89, random_state=0)),
                             ('knn',
                              KNeighborsClassifier(metric='euclidean',
                                                   n_neighbors=7)),
                             ('svc', SVC(gamma='auto', random_state=0))])

LogisticRegression(C=89, random_state=0)

In [97]:
X_t= np.zeros((len(x_test), 3))
z=0
for i in x_test:
    X_t[z, :]= extract_features(i, freqs)
    z+=1
Y_t=y_test

In [106]:
for clf in (log_clf,svm_clf, knn_clf, voting_clf):
    clf.fit(X, Y)
    y_pred = clf.predict(X_t)
    print(clf.__class__.__name__, accuracy_score(Y_t, y_pred))


LogisticRegression 0.6593939393939394
SVC 0.9346969696969697
KNeighborsClassifier 0.7206060606060606
VotingClassifier 0.8254545454545454


In [107]:
confusion_matrix(Y_t,y_pred)

array([[3619,  400],
       [ 752, 1829]], dtype=int64)

In [109]:
print(classification_report(Y_t,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86      4019
           1       0.82      0.71      0.76      2581

    accuracy                           0.83      6600
   macro avg       0.82      0.80      0.81      6600
weighted avg       0.83      0.83      0.82      6600

